In [ ]:
# import xlearn as xl

# ffm_model = xl.create_ffm() 
# ffm_model.setTrain("../dataset/all/train.csv") 
# param = {'task':'binary', 'lr':0.2, 'lambda':0.002}

# # Train model
# ffm_model.fit(param, "model.out")


In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

train_path = "../dataset/all/train.csv"
df = pd.read_csv(train_path,index_col="ID_code")

test_path = "../dataset/all/test.csv"
df_eval = pd.read_csv(test_path,index_col="ID_code")

dfLabel = df['target']
df = df.drop('target',axis=1)

dfEvalLabel = df_eval


msk = np.random.rand(len(df)) < 0.8
train = df[msk]
train_label = dfLabel[msk]
test = df[~msk]
test_label = dfLabel[~msk]



In [2]:
#le = preprocessing.LabelEncoder()
#dfLabel = le.fit(df['target']).transform(df['label'])

from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import auc

tree_model = XGBClassifier(
silent=1 ,#设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息。
#nthread=4,# cpu 线程数 默认最大
learning_rate= 0.01, # 如同学习率
min_child_weight=1,
nthread=4,  # 并行数
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
max_depth=5, # 构建树的深度，越大越容易过拟合
gamma=0.1,  # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子，对叶节点个数的惩罚系数。
subsample=1, # 随机采样训练样本 训练实例的子采样比
max_delta_step=0,#最大增量步长，我们允许每个树的权重估计。
colsample_bytree=1, # 生成树时进行的列采样
reg_lambda=1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
#reg_alpha=0, # L1 正则项参数
#scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
#objective= 'multi:softmax', #多分类的问题 指定学习任务和相应的学习目标
#num_class=10, # 类别数，多分类与 multisoftmax 并用
n_estimators=100, #树的个数
seed=1000 #随机种子
#eval_metric= 'auc'
)

tree_model.fit(train,train_label )


KeyboardInterrupt: 

In [7]:
def eval_res(test_label,preds):
	y = test_label
	scores = preds
	y_pred = (scores >= 0.5)*1
	print('AUC: %.4f' % metrics.roc_auc_score(y,scores))
	print('ACC: %.4f' % metrics.accuracy_score(y,y_pred))
	print('Recall: %.4f' % metrics.recall_score(y,y_pred))
	print('F1-score: %.4f' %metrics.f1_score(y,y_pred))
	print('Precesion: %.4f' %metrics.precision_score(y,y_pred))
    
preds = tree_model.predict_proba(test)   
eval_res(test_label,preds[:,1])


AUC: 0.7108
ACC: 0.9014
Recall: 0.0000
F1-score: 0.0000
Precesion: 0.0000


In [8]:
train_oh = tree_model.apply(train)
tm_enc = OneHotEncoder(categories='auto',sparse = 'false')
res_date = tm_enc.fit(train_oh).transform(train_oh)


In [59]:
res_date.getcol

<bound method csr_matrix.getcol of <160058x3169 sparse matrix of type '<type 'numpy.float64'>'
	with 16005800 stored elements in Compressed Sparse Row format>>

In [9]:
test_oh = tree_model.apply(test)
test_date = tm_enc.transform(test_oh)

In [10]:
res_date

<160058x3169 sparse matrix of type '<type 'numpy.float64'>'
	with 16005800 stored elements in Compressed Sparse Row format>

In [34]:
import numpy as np
###### 如果加入dense vector会强烈降低auc
# train_add = np.hstack((train.values,res_date.toarray()))
# test_add = np.hstack((test.values,test_date.toarray()))

In [48]:
import sklearn.datasets 
sklearn.datasets.dump_svmlight_file(res_date.toarray() ,train_label ,"aa.libsvm")



In [49]:
sklearn.datasets.dump_svmlight_file(test_date.toarray() ,test_label ,"bb.libsvm")

In [50]:
import xlearn as xl

ffm_model1 = xl.create_fm() 
ffm_model1.setTrain("aa.libsvm") 
ffm_model1.setValidate("bb.libsvm")

ffm_model2 = xl.create_fm() 
ffm_model2.setTrain("aa.libsvm") 
ffm_model2.setValidate("bb.libsvm")

ffm_model3 = xl.create_fm() 
ffm_model3.setTrain("aa.libsvm") 
ffm_model3.setValidate("bb.libsvm")

In [51]:
param1 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'k':8,'opt':'sgd'}
param2 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'k':8,'opt':'adagrad'}
param3 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'k':8,'opt':'ftrl'}


# Train model
ffm_model1.fit(param1, "model1.out")
ffm_model2.fit(param2, "model2.out")
ffm_model3.fit(param3, "model3.out")


In [52]:
# ffm_model.setSign()
# ffm_model.setTest("bb.libsvm")
# ffm_model.predict("model.out", "output_label.txt")

ffm_model1.setSigmoid()
ffm_model1.setTest("bb.libsvm")
ffm_model1.predict("model1.out", "output1.txt")

ffm_model2.setSigmoid()
ffm_model2.setTest("bb.libsvm")
ffm_model2.predict("model2.out", "output2.txt")

ffm_model3.setSigmoid()
ffm_model3.setTest("bb.libsvm")
ffm_model3.predict("model3.out", "output3.txt")



In [53]:
import numpy
preds = numpy.loadtxt('output1.txt')
eval_res(test_label,preds)

AUC: 0.7467
ACC: 0.9041
Recall: 0.0851
F1-score: 0.1489
Precesion: 0.5961


In [54]:
import numpy
preds = numpy.loadtxt('output2.txt')
eval_res(test_label,preds)

AUC: 0.7474
ACC: 0.9041
Recall: 0.0782
F1-score: 0.1386
Precesion: 0.6063


In [55]:
import numpy
preds = numpy.loadtxt('output.txt')
eval_res(test_label,preds)

AUC: 0.7521
ACC: 0.9037
Recall: 0.0879
F1-score: 0.1525
Precesion: 0.5748
